In [1]:
%load_ext autoreload
%autoreload 2
import time
import multiprocessing
from pathlib import Path
import shutil
import numpy as np
import thor.data as data
import thor.data.dispatch as dispatch
import thor.grid as grid
import thor.option as option
import thor.track as track
import thor.analyze as analyze
import thor.parallel as parallel
import thor.visualize as visualize
import thor.log as log

notebook_name = "gridrad_demo.ipynb"


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [11]:
# Parent directory for saving outputs
base_local = Path.home() / "THOR_output"
year=2010
event_directories = data.gridrad.get_event_directories(year, base_local=base_local)
event_directory = event_directories[0]
start, end, event_start = data.gridrad.get_event_times(event_directory)
start = "2010-01-20T22:00:00"
end = "2010-01-21T02:00:00"

period = parallel.get_period(start, end)
intervals = parallel.get_time_intervals(start, end, period=period)

output_parent = base_local / f"runs/gridrad_demo"
event_start

'2010-01-20'

In [12]:
# if output_parent.exists():
# shutil.rmtree(output_parent)
options_directory = output_parent / "options"

# Create and save the dataset options
times_dict = {"start": start, "end": end}
gridrad_dict = {"event_start": event_start}
gridrad_options = data.option.GridRadSevereOptions(**times_dict, **gridrad_dict)
era5_dict = {"latitude_range": [27, 39], "longitude_range": [-102, -89]}
era5_pl_options = data.option.ERA5Options(**times_dict, **era5_dict)
era5_dict.update({"data_format": "single-levels"})
era5_sl_options = data.option.ERA5Options(**times_dict, **era5_dict)
data_options = data.option.DataOptions(
    datasets=[gridrad_options, era5_pl_options, era5_sl_options]
)
data_options.to_yaml(options_directory / "data.yml")
gridrad_options = data_options.dataset_by_name("gridrad")

# Create and save the grid_options dictionary 
grid_options = grid.GridOptions(
    name="geographic", regrid=False, altitude_spacing=None, geographic_spacing=None
)
grid_options.to_yaml(options_directory / "grid.yml")

# Create the track_options dictionary
track_options = option.default_track_options(dataset="gridrad")
# Modify the default options for gridrad. Because grids so large we now use a distinct
# global flow box for each object.
track_options.levels[1].objects[0].tracking.global_flow_margin = 70
track_options.levels[1].objects[0].tracking.unique_global_flow = False
# If testing, remove the profile and tag attributes
# track_options.levels[1].objects[0].attributes["mcs"].pop("profile")
# track_options.levels[1].objects[0].attributes["mcs"].pop("tag")
track_options.to_yaml(options_directory / "track.yml")

# Create the display_options dictionary
# visualize_options = {
#     obj: visualize.option.runtime_options(obj, save=True, style="presentation")
#     for obj in ["mcs"]
# }
visualize_options = None

2024-11-20 10:50:03,270 - thor.data.option - INFO - Generating era5 filepaths.
2024-11-20 10:50:03,274 - thor.data.option - INFO - Generating era5 filepaths.
2024-11-20 10:50:03,277 - thor.data.option - INFO - Generating era5 filepaths.
2024-11-20 10:50:03,291 - thor.grid - WARNING - altitude_spacing not specified. Will attempt to infer frominput.
2024-11-20 10:50:03,292 - thor.grid - WARNING - shape not specified. Will attempt to infer from input.


In [7]:
times = data.utils.generate_times(data_options.dataset_by_name("gridrad"))
args = [times, data_options.model_copy(), grid_options.copy()]
args += [track_options.model_copy(), visualize_options]
track.simultaneous_track(*args, output_directory=output_parent)

2024-11-04 17:33:05,142 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/gridrad_severe/gridrad_20100120.
2024-11-04 17:33:05,144 - thor.track - INFO - Beginning simultaneous tracking.
2024-11-04 17:33:12,309 - thor.track - INFO - Processing 2010-01-20T18:00:00.
2024-11-04 17:33:12,312 - thor.data.gridrad - INFO - Updating gridrad dataset for 2010-01-20T18:00:00.
2024-11-04 17:33:12,313 - thor.data.gridrad - INFO - Converting gridrad data from nexrad_3d_v4_2_20100120T180000Z.nc
2024-11-04 17:33:13,536 - thor.track - INFO - Processing hierarchy level 0.
2024-11-04 17:33:13,537 - thor.track - INFO - Tracking convective.
2024-11-04 17:33:29,496 - thor.track - INFO - Tracking middle.
2024-11-04 17:33:29,562 - thor.track - INFO - Tracking anvil.
2024-11-04 17:33:29,614 - thor.track - INFO - Processing hierarchy level 1.
2024-11-04 17:33:29,617 - thor.track - INFO - Tracking mcs.
2024-11-04 17:33:29,807 - thor.match.match - INFO - Matching mcs objects.
2

KeyboardInterrupt: 

In [13]:
# num_processes = int(0.75 * os.cpu_count())
# num_processes = os.cpu_count()
num_processes = 4
kwargs = {"initializer": parallel.initialize_process, "processes": num_processes}
with log.logging_listener(), multiprocessing.get_context("spawn").Pool(**kwargs) as pool:
    results = []
    for i, time_interval in enumerate(intervals):
        time.sleep(1)
        args = [i, time_interval, data_options.model_copy(), grid_options.copy()]
        args += [track_options.model_copy(), visualize_options]
        args += [output_parent, "gridrad"]
        args = tuple(args)
        results.append(pool.apply_async(parallel.track_interval, args))
    pool.close()
    pool.join()
    parallel.check_results(results)


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather 

2024-11-20 10:50:18,495 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/gridrad_demo/interval_0.
2024-11-20 10:50:18,496 - thor.track - INFO - Beginning simultaneous tracking.
2024-11-20 10:50:18,565 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/gridrad_demo/interval_1.
2024-11-20 10:50:18,566 - thor.track - INFO - Beginning simultaneous tracking.
2024-11-20 10:50:18,974 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/gridrad_demo/interval_2.
2024-11-20 10:50:18,975 - thor.track - INFO - Beginning simultaneous tracking.
2024-11-20 10:50:18,983 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/gridrad_demo/interval_3.
2024-11-20 10:50:18,983 - thor.track - INFO - Beginning simultaneous tracking.
2024-11-20 10:50:19,587 - thor.track - INFO - Processing 2010-01-20T23:00:00.
2024-11-20 10:50:19,589 - thor.data.gridrad - INFO - Updating 

In [14]:
parallel.stitch_run(output_parent, intervals, cleanup=True)

2024-11-20 10:51:48,746 - thor.parallel - INFO - Stitching all attribute, mask and record files.
2024-11-20 10:51:48,749 - thor.grid - WARNING - altitude_spacing not specified. Will attempt to infer frominput.
2024-11-20 10:51:48,750 - thor.grid - WARNING - shape not specified. Will attempt to infer from input.


2024-11-20 10:51:52,277 - thor.parallel - INFO - Stitching record files.
2024-11-20 10:51:52,310 - thor.parallel - INFO - Stitching attribute files.
2024-11-20 10:51:54,015 - thor.parallel - INFO - Stitching mask files.


In [16]:
analysis_options = analyze.mcs.AnalysisOptions()
analyze.mcs.process_velocities(output_parent, profile_dataset=None)
analyze.mcs.quality_control(output_parent, analysis_options)
# analyze.mcs.classify_all(output_parent, analysis_options)

2024-11-20 10:52:30,463 - thor.grid - WARNING - altitude_spacing not specified. Will attempt to infer frominput.
2024-11-20 10:52:30,465 - thor.grid - WARNING - shape not specified. Will attempt to infer from input.


2024-11-20 10:52:30,616 - thor.grid - WARNING - altitude_spacing not specified. Will attempt to infer frominput.
2024-11-20 10:52:30,616 - thor.grid - WARNING - shape not specified. Will attempt to infer from input.


convective_contained  anvil_contained  \
time                universal_id                                          
2010-01-20 22:00:00 1                             True            False   
                    2                             True             True   
                    3                             True             True   
                    4                             True             True   
                    5                             True             True   
...                                                ...              ...   
2010-01-21 01:50:00 11                            True             True   
                    14                            True             True   
                    15                           False            False   
                    27                            True             True   
                    30                            True             True   

                                  initially_contained  velocity  area  offset  \
time                universal_id                                                
2010-01-20 22:00:00 1                           False      True  True    True   
                    2                            True      True  True    True   
                    3                            True      True  True   False   
                    4                            True      True  True    True   
                    5                            True      True  True    True   
...                                               ...       ...   ...     ...   
2010-01-21 01:50:00 11                           True      True  True    True   
                    14                           True      True  True   False   
                    15                           True      True  True    True   
                    27                           True      True  True   False   
                    30                           True      True  True   False   

                                  major_axis  axis_ratio  duration  parents  \
time                universal_id                                              
2010-01-20 22:00:00 1                   True        True      True    False   
                    2                   True        True     False    False   
                    3                   True        True     False    False   
                    4                   True       False      True     True   
                    5                   True        True     False    False   
...                                      ...         ...       ...      ...   
2010-01-21 01:50:00 11                  True        True      True     True   
                    14                  True        True      True     True   
                    15                 False       False      True     True   
                    27                  True       False      True    False   
                    30                  True       False     False    False   

                                  children  
time                universal_id            
2010-01-20 22:00:00 1                 True  
                    2                 True  
                    3                False  
                    4                 True  
                    5                 True  
...                                    ...  
2010-01-21 01:50:00 11                True  
                    14               False  
                    15                True  
                    27               False  
                    30               False  

[216 rows x 11 columns]

In [20]:
figure_name = f"mcs_gridrad_{event_start.replace('-', '')}"
figure_options = visualize.option.horizontal_attribute_options(
    figure_name, style="presentation", attributes=["velocity", "offset"]
)
start_time = np.datetime64(start)
end_time = np.datetime64(end)
args = [output_parent, start_time, end_time, figure_options]
args_dict = {"parallel_figure": True, "dt": 7200, "by_date": False, "num_processes": 4}
visualize.attribute.mcs_series(*args, **args_dict)

2024-11-20 11:05:11,243 - thor.grid - WARNING - altitude_spacing not specified. Will attempt to infer frominput.


2024-11-20 11:05:11,244 - thor.grid - WARNING - shape not specified. Will attempt to infer from input.
2024-11-20 11:05:11,511 - thor.visualize.attribute - INFO - Visualizing MCS at time 2010-01-20T22:00:00.000000000.
2024-11-20 11:05:13,056 - thor.grid - WARNING - altitude_spacing not specified. Will attempt to infer frominput.
2024-11-20 11:05:13,058 - thor.grid - WARNING - shape not specified. Will attempt to infer from input.
2024-11-20 11:05:15,564 - thor.visualize.attribute - INFO - Saving mcs_gridrad_20100120 figure for 2010-01-20T22:00:00.000000000.



## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather 

2024-11-20 11:05:24,761 - thor.visualize.attribute - INFO - Visualizing MCS at time 2010-01-20T22:10:00.000000000.
2024-11-20 11:05:24,832 - thor.visualize.attribute - INFO - Visualizing MCS at time 2010-01-20T22:20:00.000000000.
2024-11-20 11:05:25,732 - thor.visualize.attribute - INFO - Visualizing MCS at time 2010-01-20T22:30:00.000000000.
2024-11-20 11:05:27,822 - thor.visualize.attribute - INFO - Visualizing MCS at time 2010-01-20T22:40:00.000000000.
2024-11-20 11:05:28,172 - thor.grid - WARNING - altitude_spacing not specified. Will attempt to infer frominput.
2024-11-20 11:05:28,173 - thor.grid - WARNING - shape not specified. Will attempt to infer from input.
2024-11-20 11:05:28,233 - thor.grid - WARNING - altitude_spacing not specified. Will attempt to infer frominput.
2024-11-20 11:05:28,233 - thor.grid - WARNING - shape not specified. Will attempt to infer from input.
2024-11-20 11:05:29,817 - thor.grid - WARNING - altitude_spacing not specified. Will attempt to infer fromin